## Using lifespan models to make predictions on new data

This notebook shows how to apply the coefficients from [pre-estimated normative models](https://www.biorxiv.org/content/10.1101/2021.08.08.455487v2) to new data. This can be done in two different ways: (i) using a new set of data derived from the same sites used to estimate the model and (ii) on a completely different set of sites. In the latter case, we also need to estimate the site effect, which requires some calibration/adaptation data. As an illustrative example, we use a dataset derived from the [1000 functional connectomes project](https://www.nitrc.org/forum/forum.php?thread_id=2907&forum_id=1383) and adapt the learned model to make predictions on these data. 

First, if necessary, we install PCNtoolkit (note: this tutorial requires at least version 0.20)

Import the required libraries

In [ ]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import seaborn as sns

from pcntoolkit.normative import estimate, predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix
from nm_utils import remove_bad_subjects, load_2d

Next, we configure some basic variables, like where we want the analysis to be done and which model we want to use.

**Note:** We maintain a list of site ids for each dataset, which describe the site names in the training and test data (`site_ids_tr` and `site_ids_te`), plus also the adaptation data . The training site ids are provided as a text file in the distribution and the test ids are extracted automatically from the pandas dataframe (see below). If you use additional data from the sites (e.g. later waves from ABCD), it may be necessary to adjust the site names to match the names in the training set. See the accompanying [paper](https://www.biorxiv.org/content/10.1101/2021.08.08.455487v2) for more details.

In [ ]:
site_names = ['S11025', 'S11026', 'S11027']

# where the analysis takes place
root_dir = '/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master//results'

# when testing for area
out_dir = os.path.join(root_dir, 'area', 'dummy_sex')

# when testing for volume
#out_dir = os.path.join(root_dir, 'volume', 'dummy_sex')

### Load test data

Now we load the test data and remove some subjects that may have poor scan quality. This asssesment is based on the Freesurfer Euler characteristic as described in the papers below. 

**Note:** For the purposes of this tutorial, we make predictions for all sites in the FCON 1000 dataset, but two of them were also included in the training data (named 'Baltimore' and 'NewYork_a'). In this case, this will only slightly bias the accuracy, but in order to replicate the results in the paper, it would be necessary to additionally remove these sites from the test dataframe.

**References**
- [Kia et al 2021](https://www.biorxiv.org/content/10.1101/2021.05.28.446120v1.abstract)
- [Rosen et al 2018](https://www.sciencedirect.com/science/article/abs/pii/S1053811917310832?via%3Dihub)

In [ ]:
# running for control group/area
test_data = os.path.join('/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/area_inputs', 'test_data.csv')
carriers_data = os.path.join('/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/area_inputs', 'carriers_data.csv')

# running for control group/volume
#test_data = os.path.join('/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/volume_inputs', 'test_data.csv')
#carriers_data = os.path.join('/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/volume_inputs', 'carriers_data.csv')

df_te = pd.read_csv(test_data, index_col=0)
df_ca = pd.read_csv(carriers_data, index_col=0)

# remove some bad subjects
df_te, bad_sub_te = remove_bad_subjects(df_te, df_te)
df_ca, bad_sub_ca = remove_bad_subjects(df_ca, df_ca)

# extract a list of unique site ids from the test set
site_ids_te =  sorted(set(df_te['site'].to_list()))
site_ids_tr = site_ids_te

df_te contains our test set which is made of 1658 indivudals who don't carry the variants we're looking at. They are matched for age, sex and site with the carriers group.
df_ca is our carriers group: all the individuals carrying a variant.

In [ ]:
# just checking
site_ids_tr

In [ ]:
df_te = pd.get_dummies(df_te, columns=['sex'])
df_ca = pd.get_dummies(df_ca, columns=['sex'])

In [ ]:
df_te

### Configure which models to fit

Now, we configure which imaging derived phenotypes (IDPs) we would like to process. This is just a list of column names in the dataframe we have loaded above. 

We could load the whole set i.e., all phenotypes for which we have models for (188 brain regions).

In [ ]:
idp_ids = list(df_te.loc[:,df_te.columns.str.endswith('_area_z_scored')].columns)
#idp_ids = list(df_te.loc[:,df_te.columns.str.endswith('_volume_z_scored')].columns)


In [ ]:
print(idp_ids)

### Configure covariates 

Now, we configure some parameters to fit the model. First, we choose which columns of the pandas dataframe contain the covariates (age and sex). The site parameters are configured automatically later on by the `configure_design_matrix()` function, when we loop through the IDPs in the list

The supplied coefficients are derived from a 'warped' Bayesian linear regression model, which uses a nonlinear warping function to model non-Gaussianity (`sinarcsinh`) plus a non-linear basis expansion (a cubic b-spline basis set with 5 knot points, which is the default value in the PCNtoolkit package). Since we are sticking with the default value, we do not need to specify any parameters for this, but we do need to specify the limits. We choose to pad the input by a few years either side of the input range. We will also set a couple of options that control the estimation of the model

For further details about the likelihood warping approach, see the accompanying paper and [Fraza et al 2021](https://www.biorxiv.org/content/10.1101/2021.04.05.438429v1).

In [ ]:
# which data columns do we wish to use as covariates? 
#cols_cov = ['age', 'sex_0', 'sex_1', 'total_volume']
cols_cov = ['age', 'sex_0', 'sex_1', 'total_area']

# limits for cubic B-spline basis 
xmin = 45 
xmax = 80

# Absolute Z treshold above which a sample is considered to be an outlier (without fitting any model)
outlier_thresh = 7

### Make predictions

This will make predictions for each IDP separately. This is done by extracting a column from the dataframe (i.e. specifying the IDP as the response variable) and saving it as a numpy array. Then, we configure the covariates, which is a numpy data array having the number of rows equal to the number of datapoints in the test set. The columns are specified as follows: 

- A global intercept (column of ones)
- The covariate columns (here age and sex, coded as 0=female/1=male)
- Dummy coded columns for the sites in the training set (one column per site)
- Columns for the basis expansion (seven columns for the default parameterisation)

Once these are saved as numpy arrays in ascii format (as here) or (alternatively) in pickle format, these are passed as inputs to the `predict()` method in the PCNtoolkit normative modelling framework. These are written in the same format to the location specified by `idp_dir`. At the end of this step, we have a set of predictions and Z-statistics for the test dataset that we can take forward to further analysis.

Note that when we need to make predictions on new data, the procedure is more involved, since we need to prepare, process and store covariates, response variables and site ids for the adaptation data. 

In [ ]:
for idp_num, idp in enumerate(idp_ids): 
    print('Running IDP', idp_num, idp, ':')
    idp_dir = os.path.join(out_dir, idp)
    os.chdir(idp_dir)
    
    # extract and save the response variables for the test set
    y_te = df_te[idp].to_numpy()
    
    # save the variables
    resp_file_te = os.path.join(idp_dir, 'resp_te.txt') 
    np.savetxt(resp_file_te, y_te)
        
    # configure and save the design matrix
    cov_file_te = os.path.join(idp_dir, 'cov_bspline_te.txt')
    X_te = create_design_matrix(df_te[cols_cov], 
                                site_ids = df_te['site'],
                                all_sites = site_ids_tr,
                                basis = 'bspline', 
                                xmin = xmin, 
                                xmax = xmax)
    np.savetxt(cov_file_te, X_te)
    
    # check whether all sites in the test set are represented in the training set
        
    # just make predictions
    yhat_te, s2_te, Z = predict(cov_file_te, 
                                alg='blr', 
                                respfile=resp_file_te, 
                                model_path=os.path.join(idp_dir,'Models'))



In [ ]:
for idp_num, idp in enumerate(idp_ids): 
    print('Running IDP', idp_num, idp, ':')
    idp_dir = os.path.join(out_dir, idp)
    os.chdir(idp_dir)
    
    # extract and save the response variables for the test set
    y_ca = df_ca[idp].to_numpy()
    
    # save the variables
    resp_file_ca = os.path.join(idp_dir, 'resp_ca.txt') 
    np.savetxt(resp_file_ca, y_ca)
        
    # configure and save the design matrix
    cov_file_ca = os.path.join(idp_dir, 'cov_bspline_ca.txt')
    X_ca = create_design_matrix(df_ca[cols_cov], 
                                site_ids = df_ca['site'],
                                all_sites = site_ids_tr,
                                basis = 'bspline', 
                                xmin = xmin, 
                                xmax = xmax)
    np.savetxt(cov_file_ca, X_ca)
    
    # check whether all sites in the test set are represented in the training set
        
    # just make predictions
    yhat_ca, s2_ca, Z = predict(cov_file_ca, 
                                alg='blr', 
                                respfile=resp_file_ca, 
                                model_path=os.path.join(idp_dir,'Models'),
                               outputsuffix='_carriers')


### Preparing dummy data for plotting

Now, we plot the centiles of variation estimated by the normative model. 

We do this by making use of a set of dummy covariates that span the whole range of the input space (for age) for a fixed value of the other covariates (e.g. sex) so that we can make predictions for these dummy data points, then plot them. We configure these dummy predictions using the same procedure as we used for the real data. We can use the same dummy data for all the IDPs we wish to plot

In [ ]:
# which sex do we want to plot? 
sex = 1 # 1 = male 0 = female
if sex == 1: 
    clr = 'blue';
else:
    clr = 'm'

# create dummy data for visualisation
print('configuring dummy data ...')
xx = np.arange(xmin, xmax, 0.5)
total = np.linspace(min(df_te['total_area']), max(df_te['total_area']), 72)
#total = np.linspace(min(df_te['total_volume']), max(df_te['total_volume']), 72)
X0_dummy = np.zeros((len(xx), 4))
X0_dummy[:,0] = xx
X0_dummy[:,2] = sex
#X0_dummy[:,1] = sex-1
X0_dummy[:,1] = sex+1
X0_dummy[:,3] = total

In [ ]:
# create the design matrix
X_dummy = create_design_matrix(X0_dummy, xmin=xmin, xmax=xmax, site_ids=None, all_sites=site_ids_tr)

# save the dummy covariates
cov_file_dummy = os.path.join(out_dir,'cov_bspline_dummy_mean.txt')
np.savetxt(cov_file_dummy, X_dummy)

### Plotting the normative models

Now we loop through the IDPs, plotting each one separately. The outputs of this step are a set of quantitative regression metrics for each IDP and a set of centile curves which we plot the test data against. 

This part of the code is relatively complex because we need to keep track of many quantities for the plotting. We also need to remember whether the data need to be warped or not. By default in PCNtoolkit, predictions in the form of `yhat, s2` are always in the warped (Gaussian) space. If we want predictions in the input (non-Gaussian) space, then we need to warp them with the inverse of the estimated warping function. This can be done using the function `nm.blr.warp.warp_predictions()`. 

**Note:** it is necessary to update the intercept for each of the sites. For purposes of visualisation, here we do this by adjusting the median of the data to match the dummy predictions, but note that all the quantitative metrics are estimated using the predictions that are adjusted properly using a learned offset (or adjusted using a hold-out adaptation set, as above). Note also that for the calibration data we require at least two data points of the same sex in each site to be able to estimate the variance. Of course, in a real example, you would want many more than just two since we need to get a reliable estimate of the variance for each site. 

In [ ]:
sns.set(style='whitegrid')

for idp_num, idp in enumerate(idp_ids): 
    print('Running IDP', idp_num, idp, ':')
    idp_dir = os.path.join(out_dir, idp)
    os.chdir(idp_dir)
    
    # load the true data points
    yhat_te = load_2d(os.path.join(idp_dir, 'yhat_predict.txt'))
    s2_te = load_2d(os.path.join(idp_dir, 'ys2_predict.txt'))
    y_te = load_2d(os.path.join(idp_dir, 'resp_te.txt'))
    
    # load the carriers data points 
    yhat_ca = load_2d(os.path.join(idp_dir, 'yhat_carriers.txt'))
    s2_ca = load_2d(os.path.join(idp_dir, 'ys2_carriers.txt'))
    y_ca = load_2d(os.path.join(idp_dir, 'resp_ca.txt'))
            
    # set up the covariates for the dummy data
    print('Making predictions with dummy covariates (for visualisation)')
    yhat, s2 = predict(cov_file_dummy, 
                       alg = 'blr', 
                       respfile = None, 
                       model_path = os.path.join(idp_dir,'Models'), 
                       outputsuffix = '_dummy')
    
    # load the normative model
    with open(os.path.join(idp_dir,'Models', 'NM_0_0_estimate.pkl'), 'rb') as handle:
        nm = pickle.load(handle) 
    
    # get the warp and warp parameters
    W = nm.blr.warp
    warp_param = nm.blr.hyp[1:nm.blr.warp.get_n_params()+1] 
        
    # first, we warp predictions for the true data and compute evaluation metrics
    med_te = W.warp_predictions(np.squeeze(yhat_te), np.squeeze(s2_te), warp_param)[0]
    med_te = med_te[:, np.newaxis]
    print('metrics:', evaluate(y_te, med_te))
    
    med_ca = W.warp_predictions(np.squeeze(yhat_ca), np.squeeze(s2_ca), warp_param)[0]
    med_ca = med_ca[:, np.newaxis]
    print('metrics:', evaluate(y_ca, med_ca))
    
    # then, we warp dummy predictions to create the plots
    med, pr_int = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param)
    
    # extract the different variance components to visualise
    beta, junk1, junk2 = nm.blr._parse_hyps(nm.blr.hyp, X_dummy)
    s2n = 1/beta # variation (aleatoric uncertainty)
    s2s = s2-s2n # modelling uncertainty (epistemic uncertainty)
    
    # plot the data points
    y_te_rescaled_all = np.zeros_like(y_te)
    y_ca_rescaled_all = np.zeros_like(y_ca)
    for sid, site in enumerate(site_ids_te):
        # plot the true test data points 
        if all(elem in site_ids_tr for elem in site_ids_te):
            # all data in the test set are present in the training set
            
            # first, we select the data points belonging to this particular site
            idx_te = np.where(np.bitwise_and(X_te[:,3] == sex, X_te[:,sid+len(cols_cov)+1] !=0))[0]
            idx_ca = np.where(np.bitwise_and(X_ca[:,3] == sex, X_ca[:,sid+len(cols_cov)+1] !=0))[0]
            
            # then directly adjust the data
            idx_dummy_te = np.bitwise_and(X_dummy[:,1] > X_te[idx_te,1].min(), X_dummy[:,1] < X_te[idx_te,1].max())
            y_te_rescaled = y_te[idx_te] - np.median(y_te[idx_te]) + np.median(med[idx_dummy_te])
            
            idx_dummy_ca = np.bitwise_and(X_dummy[:,1] > X_ca[idx_ca,1].min(), X_dummy[:,1] < X_ca[idx_ca,1].max())
            y_ca_rescaled = y_ca[idx_ca] - np.median(y_ca[idx_ca]) + np.median(med[idx_dummy_ca])
        else:
            
            # first, select the data point belonging to this particular site
            idx = np.where(np.bitwise_and(X_te[:,2] == sex, (df_te['site'] == site).to_numpy()))[0]
            
            # load the adaptation data
            y_ad = load_2d(os.path.join(idp_dir, 'resp_ad.txt'))
            X_ad = load_2d(os.path.join(idp_dir, 'cov_bspline_ad.txt'))
            idx_a = np.where(np.bitwise_and(X_ad[:,2] == sex, (df_ad['site'] == site).to_numpy()))[0]
            if len(idx) < 2 or len(idx_a) < 2:
                print('Insufficent data for site', sid, site, 'skipping...')
                continue
            
            # adjust and rescale the data
            y_te_rescaled, s2_rescaled = nm.blr.predict_and_adjust(nm.blr.hyp, 
                                                                   X_ad[idx_a,:], 
                                                                   np.squeeze(y_ad[idx_a]), 
                                                                   Xs=None, 
                                                                   ys=np.squeeze(y_te[idx]))
        # plot the (adjusted) data points
        plt.scatter(X_te[idx_te,1], y_te_rescaled, s=4, color=clr, alpha = 0.4)
        plt.scatter(X_ca[idx_ca,1], y_ca_rescaled, s=4, color='red', alpha = 1)
       
    # plot the median of the dummy data
    plt.plot(xx, med, clr)
    
    # fill the gaps in between the centiles
    junk, pr_int25 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.05,0.95])
    junk, pr_int99 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.01,0.99])
    plt.fill_between(xx, pr_int25[:,0], pr_int25[:,1], alpha = 0.1,color=clr)
    plt.fill_between(xx, pr_int95[:,0], pr_int95[:,1], alpha = 0.1,color=clr)
    plt.fill_between(xx, pr_int99[:,0], pr_int99[:,1], alpha = 0.1,color=clr)
            
    # make the width of each centile proportional to the epistemic uncertainty
    junk, pr_int25l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.05,0.95])
    junk, pr_int99l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.01,0.99])
    junk, pr_int25u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.05,0.95])
    junk, pr_int99u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.01,0.99])    
    plt.fill_between(xx, pr_int25l[:,0], pr_int25u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int95l[:,0], pr_int95u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int99l[:,0], pr_int99u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int25l[:,1], pr_int25u[:,1], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int95l[:,1], pr_int95u[:,1], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int99l[:,1], pr_int99u[:,1], alpha = 0.3,color=clr)

    # plot actual centile lines
    plt.plot(xx, pr_int25[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int25[:,1],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int95[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int95[:,1],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int99[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int99[:,1],color=clr, linewidth=0.5)
    
    plt.xlabel('Age')
    plt.ylabel(idp) 
    plt.title(idp)
    plt.xlim((46,80))
    plt.savefig(os.path.join(idp_dir, str(sex) + '_centiles_' + idp + '_adj'),  bbox_inches='tight')
    plt.show()

os.chdir(out_dir)

The deviation scores are output as a text file in separate folders. We want to summarize the deviation scores across all models estimates so we can organize them into a single file, and merge the deviation scores into the original data file. 

In [ ]:
# Volume controls
os.chdir('/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/results/volume/dummy_sex')
! for i in *; do if [[ -e ${i}/Z_predict.txt ]]; then cp ${i}/Z_predict.txt deviation_scores/control/${i}_Z_predict.txt; fi; done
z_dir ='/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/results/volume/dummy_sex/deviation_scores/control'
filelist = [name for name in os.listdir(z_dir)]
filelist.remove('.DS_Store')
os.chdir(z_dir)
Z_df = pd.concat([pd.read_csv(item, names=[item[:-4]]) for item in filelist], axis=1)

df_te.reset_index(inplace=True)
Z_df['ID_MTL'] = df_te['ID_MTL']
df_te_Z = pd.merge(df_te, Z_df, on='ID_MTL')

os.chdir('/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/results/deviation_scores')
df_te_Z.to_csv('Zscores_volume_control.csv', index=False)

In [ ]:
# Volume carriers
os.chdir('/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/results/volume/dummy_sex')
! for i in *; do if [[ -e ${i}/Z_carriers.txt ]]; then cp ${i}/Z_carriers.txt deviation_scores/carriers/${i}_Z_carriers.txt; fi; done
z_dir ='/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/results/volume/dummy_sex/deviation_scores/carriers'
filelist = [name for name in os.listdir(z_dir)]
#filelist.remove('.DS_Store')
os.chdir(z_dir)
Z_df = pd.concat([pd.read_csv(item, names=[item[:-4]]) for item in filelist], axis=1)

df_ca.reset_index(inplace=True)
Z_df['ID_MTL'] = df_ca['ID_MTL']
df_te_Z = pd.merge(df_ca, Z_df, on='ID_MTL')

os.chdir('/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/results/deviation_scores')
df_te_Z.to_csv('Zscores_volume_carriers.csv', index=False)

In [ ]:
# Area controls
os.chdir('/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/results/area/dummy_sex')
! for i in *; do if [[ -e ${i}/Z_predict.txt ]]; then cp ${i}/Z_predict.txt deviation_scores/control/${i}_Z_predict.txt; fi; done
z_dir ='/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/results/area/dummy_sex/deviation_scores/control'
filelist = [name for name in os.listdir(z_dir)]
#filelist.remove('.DS_Store')
os.chdir(z_dir)
Z_df_ar_ctrl = pd.concat([pd.read_csv(item, names=[item[:-4]]) for item in filelist], axis=1)

#df_te.reset_index(inplace=True)
Z_df_ar_ctrl['ID_MTL'] = df_te['ID_MTL']
df_te_Z = pd.merge(df_te, Z_df_ar_ctrl, on='ID_MTL')

os.chdir('/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/results/deviation_scores')
df_te_Z.to_csv('Zscores_area_control.csv', index=False)

In [ ]:
# Area carriers
os.chdir('/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/results/area/dummy_sex')
! for i in *; do if [[ -e ${i}/Z_carriers.txt ]]; then cp ${i}/Z_carriers.txt deviation_scores/carriers/${i}_Z_carriers.txt; fi; done
z_dir ='/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/results/area/dummy_sex/deviation_scores/carriers'
filelist = [name for name in os.listdir(z_dir)]
#filelist.remove('.DS_Store')
os.chdir(z_dir)
Z_df_ar_var = pd.concat([pd.read_csv(item, names=[item[:-4]]) for item in filelist], axis=1)

#df_ca.reset_index(inplace=True)
Z_df_ar_var['ID_MTL'] = df_ca['ID_MTL']
df_ca_Z = pd.merge(df_ca, Z_df_ar_var, on='ID_MTL')

os.chdir('/Users/louisedry/Desktop/Normative_modelling/penetrance/YesNo/norm_mod_19oct21/CPC_ML_tutorial-master/results/deviation_scores')
df_ca_Z.to_csv('Zscores_area_carriers.csv', index=False)